In [2]:
import requests
import json 
from time import sleep, time
from pandas import json_normalize
import pandas as pd

In [3]:
# Verifica tu IP actual
def get_public_ip():
    try:
        response = requests.get("https://api.ipify.org?format=json")
        return response.json()["ip"]
    except:
        return None

current_ip = get_public_ip()
print(f"⚠️ Tu IP actual es: {current_ip}")

⚠️ Tu IP actual es: 172.182.200.129


In [ ]:

tag =  '#JJU8V9YR'

base_url = "https://api.clashroyale.com/v1"
endpoint = f"/players/{tag.replace('#', '%23')}/battlelog"
url = base_url + endpoint

#https://api.clashroyale.com/v1/players/%23JJU8V9YR/battlelog

response = requests.get(url, headers=headers)
response.raise_for_status()
#sleep(0.1)  # Pequeña pausa para evitar rate limiting
#response.json()

None


In [8]:


with open("token_2.txt", "r") as file:
    token = file.read().strip('\n')

base_url = "https://api.clashroyale.com/v1"

headers = {"Authorization": f"Bearer {token}"}

# Archivo donde guardaremos los resultados
output_file = "clanes_data.json"

# Lista de tags de clanes (ejemplo)
clan_tags = ["#L9CRC0PG", "#GG0C0VR0", "#P00CUJ2R", "#LC0GJGRV", "#YVQV2VVY", "#9JUP2U", "#RPQ2QGYC"] 

def get_clan_data(clan_tag):
    endpoint = f"/clans/{clan_tag.replace('#', '%23')}"  # Codificar el # como %23
    url = base_url + endpoint
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        sleep(0.1)  # Pequeña pausa para evitar rate limiting
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener datos del clan {clan_tag}: {e}")
        return None

def fetch_all_clans_data(tags, output_file):
    start_time = time()  # Iniciar medición de tiempo
    all_data = {}
    total_clans = len(tags)
    
    for i, tag in enumerate(tags, 1):
        print(f"Obteniendo datos del clan {tag} ({i}/{total_clans})...")
        clan_data = get_clan_data(tag)
        
        if clan_data is not None:
            all_data[tag] = clan_data
    
    # Guardar todos los datos
    with open(output_file, "w") as f:
        json.dump(all_data, f, indent=4)
    
    # Calcular tiempo transcurrido
    elapsed_time = time() - start_time
    mins, secs = divmod(elapsed_time, 60)
    
    print(f"\nProceso completado en {int(mins)} minutos y {secs:.2f} segundos")
    print(f"Datos de {len(all_data)}/{total_clans} clanes guardados en {output_file}")
    
    return all_data

#nota: Se tiene que revisar la IP del Codespace

In [9]:
fetch_all_clans_data(clan_tags, output_file)

Obteniendo datos del clan #L9CRC0PG (1/7)...
Obteniendo datos del clan #GG0C0VR0 (2/7)...
Obteniendo datos del clan #P00CUJ2R (3/7)...
Obteniendo datos del clan #LC0GJGRV (4/7)...
Obteniendo datos del clan #YVQV2VVY (5/7)...
Obteniendo datos del clan #9JUP2U (6/7)...
Obteniendo datos del clan #RPQ2QGYC (7/7)...

Proceso completado en 0 minutos y 2.10 segundos
Datos de 7/7 clanes guardados en clanes_data.json


{'#L9CRC0PG': {'tag': '#L9CRC0PG',
  'name': 'Biscy Saucers',
  'type': 'inviteOnly',
  'description': '1# Clan in Canada 🇨🇦 Better than the Rest! Very Active and Supportive! Path of Legends Focused 🏆 Full? Join biscy saucers 2!',
  'badgeId': 16000076,
  'clanScore': 90000,
  'clanWarTrophies': 1650,
  'location': {'id': 57000047,
   'name': 'Canada',
   'isCountry': True,
   'countryCode': 'CA'},
  'requiredTrophies': 9000,
  'donationsPerWeek': 4120,
  'clanChestStatus': 'inactive',
  'clanChestLevel': 1,
  'clanChestMaxLevel': 0,
  'members': 50,
  'memberList': [{'tag': '#JJU8V9YR',
    'name': 'BigDanny224',
    'role': 'coLeader',
    'lastSeen': '20250601T034849.000Z',
    'expLevel': 69,
    'trophies': 9000,
    'arena': {'id': 54000020, 'name': 'Valkalla'},
    'clanRank': 1,
    'previousClanRank': 1,
    'donations': 272,
    'donationsReceived': 0,
    'clanChestPoints': 0},
   {'tag': '#C22U88U2',
    'name': 'Bo$$',
    'role': 'coLeader',
    'lastSeen': '20250601T0242

In [10]:

# 1. Cargar el archivo correctamente
with open('clanes_data.json', 'r') as f:
    clan_data = json.load(f)  # clan_data es ahora un diccionario

# 2. Convertir a DataFrame (estructura deseada)
df = (
    pd.DataFrame.from_dict(clan_data, orient='index')
    
)

# 3. Si hay datos anidados (ej: "location": {"country": "Chile"}):
clan_df = pd.json_normalize(
    df.to_dict('records'),  # Datos a normalizar
    sep='_',
    max_level= 3                # Separador para campos anidados
)

#clan_df['diccionario'] = clan_df['memberList'].apply(lambda x: x[0])
clan_df.rename(columns={'tag': 'tag_clan'}, inplace=True)

jugadores = []
for idx, row in clan_df.iterrows():
    tag_clan = row['tag_clan']
    for miembro in row['memberList']:
        miembro['tag_clan'] = tag_clan
        jugadores.append(miembro)

# Convertir a DataFrame
jugadores_df = pd.DataFrame(jugadores)



In [11]:
clan_df.head()

,tag_clan,name,type,description,badgeId,clanScore,clanWarTrophies,requiredTrophies,donationsPerWeek,clanChestStatus,clanChestLevel,clanChestMaxLevel,members,memberList,location_id,location_name,location_isCountry,location_countryCode
0,#L9CRC0PG,Biscy Saucers,inviteOnly,1# Clan in Canada 🇨🇦 Better than the Rest! Ver...,16000076,90000,1650,9000,4120,inactive,1,0,50,"[{'tag': '#JJU8V9YR', 'name': 'BigDanny224', '...",57000047,Canada,True,CA
1,#GG0C0VR0,Jynxzi Verse,inviteOnly,,16000008,90000,180,9000,7464,inactive,1,0,50,"[{'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role'...",57000006,International,False,NaN
2,#P00CUJ2R,Eternal Imperio,open,🔱Bienvenidos a la familia Eternal🔱 ⚜️Una nueva...,16000013,90000,2300,9000,2870,inactive,1,0,50,"[{'tag': '#2VQVGJC', 'name': 'ＡＬＥＸＩＳ⁵⁰⁷', 'rol...",57000181,Panama,True,PA
3,#LC0GJGRV,los mercenarios,inviteOnly,"Se regalan 3 pass a los 3 top guerra. 2,200 pt...",16000077,90000,4922,9000,7995,inactive,1,0,50,"[{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', ...",57000169,Nicaragua,True,NI
4,#YVQV2VVY,BAD BOYS,open,♨️CLAN COMPETITIVO♨️ATACAR EN GUERRA 4/4 ATAQU...,16000009,89928,3693,9000,8123,inactive,1,0,50,"[{'tag': '#JQ82089QR', 'name': 'xoguh12', 'rol...",57000153,Mexico,True,MX


In [12]:
jugadores_df.head()

,tag,name,role,lastSeen,expLevel,trophies,arena,clanRank,previousClanRank,donations,donationsReceived,clanChestPoints,tag_clan
0,#JJU8V9YR,BigDanny224,coLeader,20250601T034849.000Z,69,9000,"{'id': 54000020, 'name': 'Valkalla'}",1,1,272,0,0,#L9CRC0PG
1,#C22U88U2,Bo$$,coLeader,20250601T024236.000Z,69,9000,"{'id': 54000020, 'name': 'Valkalla'}",2,2,316,0,0,#L9CRC0PG
2,#CG8PURUV0,弩x-bow弩,member,20250601T060110.000Z,60,9000,"{'id': 54000020, 'name': 'Valkalla'}",3,3,260,680,0,#L9CRC0PG
3,#2YYLYPQRV,moopeloo,elder,20250531T213828.000Z,62,9000,"{'id': 54000020, 'name': 'Valkalla'}",4,6,20,0,0,#L9CRC0PG
4,#PQ2LVJ,Zach,coLeader,20250531T235908.000Z,65,9000,"{'id': 54000020, 'name': 'Valkalla'}",5,4,56,0,0,#L9CRC0PG


In [62]:
user_df['diccionario']

0    {'tag': '#JJU8V9YR', 'name': 'BigDanny224', 'r...
1    {'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role':...
2    {'tag': '#20Y8GVRLQ', 'name': 'david liu', 'ro...
3    {'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', '...
4    {'tag': '#JQ82089QR', 'name': 'xoguh12', 'role...
5    {'tag': '#2GCGJU0L', 'name': 'Dhaztan', 'role'...
6    {'tag': '#Y08V0CPLR', 'name': 'allah', 'role':...
Name: diccionario, dtype: object

In [61]:
pd.json_normalize(user_df['diccionario'],  
    sep='_',
    max_level= 3)

,tag,name,role,lastSeen,expLevel,trophies,clanRank,previousClanRank,donations,donationsReceived,clanChestPoints,arena_id,arena_name
0,#JJU8V9YR,BigDanny224,coLeader,20250530T031133.000Z,69,9000,1,1,166,0,0,54000020,Valkalla
1,#C0V0UQ9UY,Ryley,coLeader,20250530T020739.000Z,67,9000,1,0,0,0,0,54000020,Valkalla
2,#20Y8GVRLQ,david liu,member,20250529T213636.000Z,66,9000,1,2,99,0,0,54000020,Valkalla
3,#9PGU890Q9,MikeXGamer™,member,20250530T044954.000Z,69,9000,1,1,37,0,0,54000020,Valkalla
4,#JQ82089QR,xoguh12,member,20250530T021929.000Z,60,9000,1,1,36,0,0,54000020,Valkalla
5,#2GCGJU0L,Dhaztan,coLeader,20250530T034106.000Z,64,9000,1,3,0,0,0,54000020,Valkalla
6,#Y08V0CPLR,allah,coLeader,20250529T202718.000Z,57,9000,1,2,346,290,0,54000020,Valkalla


In [52]:
pd.json_normalize(t['diccionario'])

,tag,name,role,lastSeen,expLevel,trophies,clanRank,previousClanRank,donations,donationsReceived,clanChestPoints,arena.id,arena.name
0,#JQ82089QR,xoguh12,member,20250530T021929.000Z,60,9000,1,1,36,0,0,54000020,Valkalla


In [37]:
pd.json_normalize(user_df['diccionario'])

,tag,name,role,lastSeen,expLevel,trophies,clanRank,previousClanRank,donations,donationsReceived,clanChestPoints,arena.id,arena.name
0,#JJU8V9YR,BigDanny224,coLeader,20250530T031133.000Z,69,9000,1,1,166,0,0,54000020,Valkalla
1,#C0V0UQ9UY,Ryley,coLeader,20250530T020739.000Z,67,9000,1,0,0,0,0,54000020,Valkalla
2,#20Y8GVRLQ,david liu,member,20250529T213636.000Z,66,9000,1,2,99,0,0,54000020,Valkalla
3,#9PGU890Q9,MikeXGamer™,member,20250530T044954.000Z,69,9000,1,1,37,0,0,54000020,Valkalla
4,#JQ82089QR,xoguh12,member,20250530T021929.000Z,60,9000,1,1,36,0,0,54000020,Valkalla
5,#2GCGJU0L,Dhaztan,coLeader,20250530T034106.000Z,64,9000,1,3,0,0,0,54000020,Valkalla
6,#Y08V0CPLR,allah,coLeader,20250529T202718.000Z,57,9000,1,2,346,290,0,54000020,Valkalla


In [54]:
jugadores_df = pd.json_normalize(user_df['diccionario'])

# Añadir la columna 'tag_clan' correspondiente
#jugadores_df['tag_clan'] = user_df['tag_clan']

jugadores_df.head()

,tag,name,role,lastSeen,expLevel,trophies,clanRank,previousClanRank,donations,donationsReceived,clanChestPoints,arena.id,arena.name
0,#JJU8V9YR,BigDanny224,coLeader,20250530T031133.000Z,69,9000,1,1,166,0,0,54000020,Valkalla
1,#C0V0UQ9UY,Ryley,coLeader,20250530T020739.000Z,67,9000,1,0,0,0,0,54000020,Valkalla
2,#20Y8GVRLQ,david liu,member,20250529T213636.000Z,66,9000,1,2,99,0,0,54000020,Valkalla
3,#9PGU890Q9,MikeXGamer™,member,20250530T044954.000Z,69,9000,1,1,37,0,0,54000020,Valkalla
4,#JQ82089QR,xoguh12,member,20250530T021929.000Z,60,9000,1,1,36,0,0,54000020,Valkalla


In [56]:
jugadores_df.shape

(7, 13)

In [33]:
user_df.head()

,diccionario,tag_clan
0,"{'tag': '#JJU8V9YR', 'name': 'BigDanny224', 'r...",#L9CRC0PG
1,"{'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role':...",#GG0C0VR0
2,"{'tag': '#20Y8GVRLQ', 'name': 'david liu', 'ro...",#P00CUJ2R
3,"{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', '...",#LC0GJGRV
4,"{'tag': '#JQ82089QR', 'name': 'xoguh12', 'role...",#YVQV2VVY


In [12]:
df_final.head(28)

,tag,name,type,description,badgeId,clanScore,clanWarTrophies,location,requiredTrophies,donationsPerWeek,...,lastSeen,expLevel,trophies,clanRank,previousClanRank,donations,donationsReceived,clanChestPoints,arena_id,arena_name
#L9CRC0PG,#L9CRC0PG,Biscy Saucers,inviteOnly,1# Clan in Canada 🇨🇦 Better than the Rest! Ver...,16000076.0,90000.0,1650.0,"{'id': 57000047, 'name': 'Canada', 'isCountry'...",9000.0,2760.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#GG0C0VR0,#GG0C0VR0,Jynxzi Verse,inviteOnly,,16000008.0,90000.0,180.0,"{'id': 57000006, 'name': 'International', 'isC...",9000.0,5024.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#P00CUJ2R,#P00CUJ2R,Eternal Imperio,open,🔱Bienvenidos a la familia Eternal🔱 ⚜️Una nueva...,16000013.0,90000.0,2300.0,"{'id': 57000181, 'name': 'Panama', 'isCountry'...",9000.0,2126.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#LC0GJGRV,#LC0GJGRV,los mercenarios,inviteOnly,"Se regalan 3 pass a los 3 top guerra. 2,200 pt...",16000077.0,90000.0,4922.0,"{'id': 57000169, 'name': 'Nicaragua', 'isCount...",9000.0,4895.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#YVQV2VVY,#YVQV2VVY,BAD BOYS,inviteOnly,♨️CLAN COMPETITIVO♨️ATACAR EN GUERRA 4/4 ATAQU...,16000009.0,89460.0,3693.0,"{'id': 57000153, 'name': 'Mexico', 'isCountry'...",9000.0,5319.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#9JUP2U,#9JUP2U,HOLLIGANS,open,clan de chill pero con ganas de competir / obl...,16000029.0,79655.0,1833.0,"{'id': 57000153, 'name': 'Mexico', 'isCountry'...",1300.0,2224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#RPQ2QGYC,#RPQ2QGYC,ONE PIECE,open,ONE PIECE İZLEYİN İZLETTİRİN!!! Klan savaşları...,16000024.0,76195.0,510.0,"{'id': 57000239, 'name': 'Turkey', 'isCountry'...",6500.0,3393.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,#JJU8V9YR,BigDanny224,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20250530T031133.000Z,69.0,9000.0,1.0,1.0,166.0,0.0,0.0,54000020.0,Valkalla
1,#C0V0UQ9UY,Ryley,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20250530T020739.000Z,67.0,9000.0,1.0,0.0,0.0,0.0,0.0,54000020.0,Valkalla
2,#20Y8GVRLQ,david liu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20250529T213636.000Z,66.0,9000.0,1.0,2.0,99.0,0.0,0.0,54000020.0,Valkalla


In [6]:
df_final.sample(7)

,tag,name,type,description,badgeId,clanScore,clanWarTrophies,location,requiredTrophies,donationsPerWeek,...,lastSeen,expLevel,trophies,clanRank,previousClanRank,donations,donationsReceived,clanChestPoints,arena_id,arena_name
#LC0GJGRV,#LC0GJGRV,los mercenarios,inviteOnly,"Se regalan 3 pass a los 3 top guerra. 2,200 pt...",16000077.0,90000.0,4922.0,"{'id': 57000169, 'name': 'Nicaragua', 'isCount...",9000.0,4895.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#GG0C0VR0,#GG0C0VR0,Jynxzi Verse,inviteOnly,,16000008.0,90000.0,180.0,"{'id': 57000006, 'name': 'International', 'isC...",9000.0,5024.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20250529T202718.000Z,57.0,9000.0,1.0,2.0,346.0,290.0,0.0,54000020.0,Valkalla
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20250530T021929.000Z,60.0,9000.0,1.0,1.0,36.0,0.0,0.0,54000020.0,Valkalla
#RPQ2QGYC,#RPQ2QGYC,ONE PIECE,open,ONE PIECE İZLEYİN İZLETTİRİN!!! Klan savaşları...,16000024.0,76195.0,510.0,"{'id': 57000239, 'name': 'Turkey', 'isCountry'...",6500.0,3393.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
#L9CRC0PG,#L9CRC0PG,Biscy Saucers,inviteOnly,1# Clan in Canada 🇨🇦 Better than the Rest! Ver...,16000076.0,90000.0,1650.0,"{'id': 57000047, 'name': 'Canada', 'isCountry'...",9000.0,2760.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20250530T031133.000Z,69.0,9000.0,1.0,1.0,166.0,0.0,0.0,54000020.0,Valkalla


In [100]:
df_final['diccionario'] = df_final['memberList'].apply(lambda x: x[0])

# Paso 2: Convertir esa columna en columnas individuales
dic_df = pd.json_normalize(df_final['diccionario'])

# Paso 3: Unir al DataFrame original si quieres mantener otras columnas
#df_final = pd.concat([df, dic_df], axis=1).drop(columns=['datos', 'diccionario'])


In [74]:
tst = df[['tag','memberList']]
tst.head()

,tag,memberList
#L9CRC0PG,#L9CRC0PG,"[{'tag': '#JJU8V9YR', 'name': 'BigDanny224', '..."
#GG0C0VR0,#GG0C0VR0,"[{'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role'..."
#P00CUJ2R,#P00CUJ2R,"[{'tag': '#20Y8GVRLQ', 'name': 'david liu', 'r..."
#LC0GJGRV,#LC0GJGRV,"[{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', ..."
#YVQV2VVY,#YVQV2VVY,"[{'tag': '#JQ82089QR', 'name': 'xoguh12', 'rol..."


In [78]:
tst['memberList'][0][0]

/tmp/ipykernel_15491/384682349.py:1: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tst['memberList'][0][0]


{'tag': '#JJU8V9YR',
 'name': 'BigDanny224',
 'role': 'coLeader',
 'lastSeen': '20250530T031133.000Z',
 'expLevel': 69,
 'trophies': 9000,
 'arena': {'id': 54000020, 'name': 'Valkalla'},
 'clanRank': 1,
 'previousClanRank': 1,
 'donations': 166,
 'donationsReceived': 0,
 'clanChestPoints': 0}

In [81]:
tst['diccionario'] = tst['memberList'].apply(lambda x: x[0])
tst.head()

/tmp/ipykernel_15491/1423004643.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tst['diccionario'] = tst['memberList'].apply(lambda x: x[0])


,tag,memberList,diccionario
#L9CRC0PG,#L9CRC0PG,"[{'tag': '#JJU8V9YR', 'name': 'BigDanny224', '...","{'tag': '#JJU8V9YR', 'name': 'BigDanny224', 'r..."
#GG0C0VR0,#GG0C0VR0,"[{'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role'...","{'tag': '#C0V0UQ9UY', 'name': 'Ryley', 'role':..."
#P00CUJ2R,#P00CUJ2R,"[{'tag': '#20Y8GVRLQ', 'name': 'david liu', 'r...","{'tag': '#20Y8GVRLQ', 'name': 'david liu', 'ro..."
#LC0GJGRV,#LC0GJGRV,"[{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', ...","{'tag': '#9PGU890Q9', 'name': 'MikeXGamer™', '..."
#YVQV2VVY,#YVQV2VVY,"[{'tag': '#JQ82089QR', 'name': 'xoguh12', 'rol...","{'tag': '#JQ82089QR', 'name': 'xoguh12', 'role..."


In [ ]:
i[0]

In [66]:
for i in df['memberList']:
    print(i.ittems())  # Imprime el nombre del primer miembro de cada clan

AttributeError: 'list' object has no attribute 'ittems'

In [ ]:
#transformando el dataframe
df_transformed = df.reset_index().rename(columns={'index': 'tag_clan'})
#normalizando columnas
df_final = json_normalize(df_transformed.to_dict('records'))

In [ ]:
df_final.info()

In [ ]:
df_final.sample(5)

In [ ]:
df_final.info()

In [ ]:
with open("token.txt", "r") as file:
    token = file.read().strip()
    print(f"Token usado: '{token}'") 

In [ ]:
import requests
import json
from time import sleep, time

# Leer el token
with open("token.txt", "r") as file:
    token = file.read().strip()

print(f"Token usado: '{token}'")  # Debug: verifica el token

base_url = "https://api.clashroyale.com/v1"
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json"
}

# Test endpoint público (para verificar token)
def test_token():
    test_url = base_url + "/cards"
    try:
        response = requests.get(test_url, headers=headers)
        print(f"Test response: {response.status_code}")
        if response.status_code == 200:
            print("¡Token válido!")
        else:
            print(f"Error: {response.json()}")
    except Exception as e:
        print(f"Error en test: {e}")

test_token()  # Ejecuta esta prueba primero

# Si el test falla, detén el programa aquí
if input("¿El token es válido? (s/n): ").lower() != 's':
    exit()